In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import re
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 
import pycaret

In [3]:
path_read ="/Users/a06411/Documents/data_hub/lending_club/lgb_selected.pkl"

In [4]:
df = pd.read_pickle(path_read)
# df.drop(['Unnamed: 0'],1,inplace=True)

In [5]:
df.shape

(1382351, 63)

In [6]:
df.head()

,int_rate,dti,annual_inc,mo_sin_old_rev_tl_op,acc_open_past_24mths,loan_amnt,emp_length,addr_state,revol_bal,term,...,total_bal_ex_mort,mths_since_last_major_derog,inq_fi,mths_since_rcnt_il,inq_last_12m,mths_since_last_delinq,num_bc_tl,loan_status,issue_d,id
0,13.99,5.91,55000.0,128.0,4.0,3600.0,1,38,2765.0,0,...,7746.0,30.0,3.0,21.0,4.0,30.0,5.0,0,Dec-2015,68407277
1,11.99,16.06,65000.0,192.0,4.0,24700.0,1,41,21470.0,0,...,39475.0,-1.0,0.0,19.0,6.0,6.0,17.0,0,Dec-2015,68355089
2,10.78,10.78,63000.0,184.0,6.0,20000.0,1,14,7869.0,1,...,18696.0,-1.0,2.0,19.0,1.0,-1.0,4.0,0,Dec-2015,68341763
4,22.45,25.37,104433.0,210.0,10.0,10400.0,3,38,21929.0,1,...,95768.0,-1.0,2.0,14.0,3.0,12.0,9.0,0,Dec-2015,68476807
5,13.44,10.20,34000.0,54.0,0.0,11950.0,4,10,8822.0,0,...,12798.0,-1.0,0.0,338.0,0.0,-1.0,2.0,0,Dec-2015,68426831


## 파이캐럿 autoML로 처리하기 

In [7]:
from pycaret.classification import *

## Make sample
- Since using original df outrange our RAM capacity
- Make sample of df for Pycaret to comparing Algorithms

In [24]:
df_sample = df.copy()

In [28]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41471 entries, 1054037 to 386477
Data columns (total 63 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   int_rate                     41471 non-null  float64
 1   dti                          41471 non-null  float64
 2   annual_inc                   41471 non-null  float64
 3   mo_sin_old_rev_tl_op         41471 non-null  float64
 4   acc_open_past_24mths         41471 non-null  float64
 5   loan_amnt                    41471 non-null  float64
 6   emp_length                   41471 non-null  int64  
 7   addr_state                   41471 non-null  int64  
 8   revol_bal                    41471 non-null  float64
 9   term                         41471 non-null  int64  
 10  sub_grade                    41471 non-null  int64  
 11  funded_amnt_inv              41471 non-null  float64
 12  installment                  41471 non-null  float64
 13  purpose  

In [25]:
df_sample = df_sample.sample(frac=0.03, random_state=20)

In [26]:
df_sample.shape

(41471, 63)

In [30]:
df_sample.columns[:-2]

Index(['int_rate', 'dti', 'annual_inc', 'mo_sin_old_rev_tl_op',
       'acc_open_past_24mths', 'loan_amnt', 'emp_length', 'addr_state',
       'revol_bal', 'term', 'sub_grade', 'funded_amnt_inv', 'installment',
       'purpose', 'total_rev_hi_lim', 'fico_range_low', 'debt_settlement_flag',
       'mort_acc', 'total_bc_limit', 'home_ownership', 'avg_cur_bal',
       'all_util', 'mths_since_recent_bc', 'total_acc', 'open_acc_6m',
       'bc_util', 'num_actv_rev_tl', 'funded_amnt', 'hardship_flag',
       'num_rev_tl_bal_gt_0', 'mths_since_recent_inq', 'inq_last_6mths',
       'num_il_tl', 'mo_sin_old_il_acct', 'num_rev_accts', 'num_tl_120dpd_2m',
       'total_il_high_credit_limit', 'application_type', 'revol_util',
       'tot_hi_cred_lim', 'delinq_2yrs', 'mo_sin_rcnt_tl', 'num_actv_bc_tl',
       'mths_since_last_record', 'percent_bc_gt_75', 'bc_open_to_buy',
       'max_bal_bc', 'grade', 'open_rv_24m', 'mo_sin_rcnt_rev_tl_op',
       'pct_tl_nvr_dlq', 'verification_status', 'tot_cur_b

## Setting up Pycaret

In [32]:
st_clf = setup(data = df_sample[df_sample.columns[:-2]], target = 'loan_status')

,Description,Value
0,Session id,3562
1,Target,loan_status
2,Target type,Binary
3,Original data shape,"(41471, 61)"
4,Transformed data shape,"(41471, 61)"
5,Transformed train set shape,"(29029, 61)"
6,Transformed test set shape,"(12442, 61)"
7,Numeric features,60
8,Preprocess,True
9,Imputation type,simple


## Comparing Models

In [33]:
best_3 = compare_models(sort ='AUC', fold=3, n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8140,0.7527,0.2031,0.7201,0.3167,0.2463,0.3132,2.5800
gbc,Gradient Boosting Classifier,0.8147,0.7513,0.1776,0.7807,0.2892,0.2286,0.3129,3.6933
lda,Linear Discriminant Analysis,0.8139,0.7487,0.1888,0.7447,0.3010,0.2354,0.3103,0.1567
lightgbm,Light Gradient Boosting Machine,0.8112,0.7459,0.1955,0.6981,0.3053,0.2342,0.2985,0.2833
ada,Ada Boost Classifier,0.8121,0.7451,0.1886,0.7204,0.2987,0.2309,0.3011,0.8167
et,Extra Trees Classifier,0.8121,0.7368,0.1603,0.7805,0.2659,0.2086,0.2965,0.4200
rf,Random Forest Classifier,0.8124,0.7354,0.1666,0.7705,0.2738,0.2145,0.2992,0.7567
xgboost,Extreme Gradient Boosting,0.8029,0.7267,0.2373,0.5900,0.3384,0.2465,0.2820,1.7767
qda,Quadratic Discriminant Analysis,0.8096,0.6862,0.1155,0.9045,0.2048,0.1646,0.2824,0.1300
nb,Naive Bayes,0.5924,0.6574,0.6527,0.2935,0.4048,0.1582,0.1875,0.4800
